In [14]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import string
import glob
import re
import random

from sklearn.preprocessing import StandardScaler
import hdbscan

In [12]:
stops_tidy = pd.read_csv('../data/stops_tidy.csv',index_col=0)
stops_tidy

,Translator,Work,Chunk,atqui,aut,autem,certe,ceu,confestim,cum,...,sic,sicut,siquidem,tamquam,ut,utique,velut,veluti,vero,videlicet
0,Bur,Com,quoniam quidem ex calido et frigido et sicco e...,0.0,0.034605,0.190328,0.0,0.0,0.0,0.051908,...,0.000000,0.000000,0.000000,0.0,0.051908,0.077862,0.008651,0.0,0.077862,0.0
1,Bur,Com,sermo non quod nunquam fit in uno eodemque cor...,0.0,0.079463,0.229559,0.0,0.0,0.0,0.061804,...,0.000000,0.000000,0.008829,0.0,0.088292,0.123609,0.008829,0.0,0.026488,0.0
2,Bur,Com,horis anni invenire quartam coniugationem comp...,0.0,0.091264,0.228161,0.0,0.0,0.0,0.000000,...,0.018253,0.000000,0.000000,0.0,0.100391,0.054759,0.018253,0.0,0.091264,0.0
3,Bur,Com,quod necesse est in ea putrefieri omnia incipi...,0.0,0.114119,0.333577,0.0,0.0,0.0,0.017557,...,0.000000,0.000000,0.000000,0.0,0.131675,0.079005,0.000000,0.0,0.017557,0.0
4,Bur,Com,de ipsis per capitula quantum ad presentia uti...,0.0,0.371354,0.234115,0.0,0.0,0.0,0.040365,...,0.000000,0.000000,0.000000,0.0,0.226042,0.121094,0.000000,0.0,0.040365,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,Bur,Fid,ostendat quod secundum veritatem est homo cum ...,0.0,0.008708,0.191573,0.0,0.0,0.0,0.060955,...,0.000000,0.000000,0.000000,0.0,0.208988,0.008708,0.000000,0.0,0.008708,0.0
1172,Bur,Fid,causative dicere ut hoc tibi soli peccavi et p...,0.0,0.000000,0.162756,0.0,0.0,0.0,0.019148,...,0.000000,0.000000,0.000000,0.0,0.086165,0.019148,0.009574,0.0,0.028722,0.0
1173,Bur,Fid,condemnavit peccatum in carne ut iustitia legi...,0.0,0.000000,0.229977,0.0,0.0,0.0,0.029997,...,0.000000,0.009999,0.000000,0.0,0.149985,0.019998,0.009999,0.0,0.009999,0.0
1174,Bur,Fid,uxorem suam et concepit et genuit quare propte...,0.0,0.000000,0.170615,0.0,0.0,0.0,0.040145,...,0.000000,0.020072,0.000000,0.0,0.070253,0.010036,0.000000,0.0,0.010036,0.0


In [22]:
tmp = stops_tidy.drop(['Translator','Work','Chunk'], axis=1)
tmp

,atqui,aut,autem,certe,ceu,confestim,cum,dehinc,deinceps,demum,...,sic,sicut,siquidem,tamquam,ut,utique,velut,veluti,vero,videlicet
0,0.0,0.034605,0.190328,0.0,0.0,0.0,0.051908,0.0,0.008651,0.0,...,0.000000,0.000000,0.000000,0.0,0.051908,0.077862,0.008651,0.0,0.077862,0.0
1,0.0,0.079463,0.229559,0.0,0.0,0.0,0.061804,0.0,0.000000,0.0,...,0.000000,0.000000,0.008829,0.0,0.088292,0.123609,0.008829,0.0,0.026488,0.0
2,0.0,0.091264,0.228161,0.0,0.0,0.0,0.000000,0.0,0.009126,0.0,...,0.018253,0.000000,0.000000,0.0,0.100391,0.054759,0.018253,0.0,0.091264,0.0
3,0.0,0.114119,0.333577,0.0,0.0,0.0,0.017557,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.131675,0.079005,0.000000,0.0,0.017557,0.0
4,0.0,0.371354,0.234115,0.0,0.0,0.0,0.040365,0.0,0.008073,0.0,...,0.000000,0.000000,0.000000,0.0,0.226042,0.121094,0.000000,0.0,0.040365,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,0.0,0.008708,0.191573,0.0,0.0,0.0,0.060955,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.208988,0.008708,0.000000,0.0,0.008708,0.0
1172,0.0,0.000000,0.162756,0.0,0.0,0.0,0.019148,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.086165,0.019148,0.009574,0.0,0.028722,0.0
1173,0.0,0.000000,0.229977,0.0,0.0,0.0,0.029997,0.0,0.000000,0.0,...,0.000000,0.009999,0.000000,0.0,0.149985,0.019998,0.009999,0.0,0.009999,0.0
1174,0.0,0.000000,0.170615,0.0,0.0,0.0,0.040145,0.0,0.000000,0.0,...,0.000000,0.020072,0.000000,0.0,0.070253,0.010036,0.000000,0.0,0.010036,0.0


In [23]:
# Since we're clustering etc now it's better to rescale the data. StandardScaler
# just does z-scaling which is not amazingly robust in the face of outliers but
# is very vanilla in terms of methodology.

X = pd.DataFrame(StandardScaler(with_mean=False).fit_transform(tmp))
X.columns = tmp.columns
X


,atqui,aut,autem,certe,ceu,confestim,cum,dehinc,deinceps,demum,...,sic,sicut,siquidem,tamquam,ut,utique,velut,veluti,vero,videlicet
0,0.0,0.553736,1.464444,0.0,0.0,0.0,1.610526,0.0,1.388188,0.0,...,0.000000,0.000000,0.000000,0.0,1.165954,2.224510,0.642272,0.0,1.429771,0.0
1,0.0,1.271527,1.766296,0.0,0.0,0.0,1.917584,0.0,0.000000,0.0,...,0.000000,0.000000,1.013534,0.0,1.983216,3.531505,0.655479,0.0,0.486391,0.0
2,0.0,1.460371,1.755539,0.0,0.0,0.0,0.000000,0.0,1.464430,0.0,...,0.788445,0.000000,0.000000,0.0,2.254982,1.564456,1.355093,0.0,1.675885,0.0
3,0.0,1.826075,2.566647,0.0,0.0,0.0,0.544727,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,2.957697,2.257183,0.000000,0.0,0.322394,0.0
4,0.0,5.942250,1.801351,0.0,0.0,0.0,1.252381,0.0,1.295384,0.0,...,0.000000,0.000000,0.000000,0.0,5.077364,3.459659,0.000000,0.0,0.741215,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,0.0,0.139339,1.474018,0.0,0.0,0.0,1.891230,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,4.694305,0.248784,0.000000,0.0,0.159902,0.0
1172,0.0,0.000000,1.252294,0.0,0.0,0.0,0.594092,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,1.935440,0.547052,0.710765,0.0,0.527415,0.0
1173,0.0,0.000000,1.769514,0.0,0.0,0.0,0.930708,0.0,0.000000,0.0,...,0.000000,0.417625,0.000000,0.0,3.368972,0.571344,0.742327,0.0,0.183612,0.0
1174,0.0,0.000000,1.312767,0.0,0.0,0.0,1.245561,0.0,0.000000,0.0,...,0.000000,0.838357,0.000000,0.0,1.578035,0.286735,0.000000,0.0,0.184295,0.0
